In [1]:
import os
import re

## Folder

In [2]:
Data_Input_Folder = "../Forms/Data_Testing/Input"
Data_Label_Folder = "../Forms/Data_Testing/Label_Output"

# Function to read file contents
def read_file(file_path):
    try:
        with open(file_path, 'r',encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"The file {file_path} does not exist.")
        return None
for index,filename in enumerate(os.listdir(Data_Label_Folder)[107:109]):
    print(filename)

49_00_TK1-TS.txt
49_01_TK1-TS.txt


In [3]:
input_form = read_file(Data_Input_Folder+'/'+os.listdir(Data_Input_Folder)[64]).replace('..........','[#another]')
label_form = read_file(Data_Label_Folder+'/'+os.listdir(Data_Label_Folder)[64]).replace('..........','[#another]')

In [4]:
# print(input_form)

In [5]:
# print(label_form)

## Re

In [6]:
input_form = """
CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
Độc lập - Tự do - Hạnh phúc
---------------
GIẤY ĐỀ NGHỊ XÁC NHẬN SỐ CHỨNG MINH NHÂN DÂN

Kính gửi(1):..........
1. Họ, chữ đệm và tên(2): ..........
2. Họ, chữ đệm và tên gọi khác (nếu có)(2): ..........
3. Ngày, tháng, năm sinh: ........../........../.......... ; 4. Giới tính (Nam/nữ): ..........
5. Dân tộc:.......... 6. Quốc tịch:..........
7. Nơi đăng ký khai sinh: ..........
8. Quê quán: ..........
9. Nơi thường trú: ..........
10. Số CCCD đã được cấp (3):.......... tại:
11. Số CMND đã được đã được cấp (4):.......... 
Đề nghị (1):.......... xác nhận số Chứng minh nhân dân và số thẻ Căn cước cước công dân cho tôi. Tôi xin cam đoan những thông tin kê khai trên là đúng sự thật.
    .........., ngày ..........tháng..........năm..........
""".replace("..........",'[#another]')

label_form = """
CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
Độc lập - Tự do - Hạnh phúc
---------------
GIẤY ĐỀ NGHỊ XÁC NHẬN SỐ CHỨNG MINH NHÂN DÂN

Kính gửi(1): [receiver]
1. Họ, chữ đệm và tên(2): [user1_alias_name]
2. Họ, chữ đệm và tên gọi khác (nếu có)(2): [user1_alias_name]
3. Ngày, tháng, năm sinh: [user1_dob_day]/[user1_dob_month]/[user1_dob_year] ; 4. Giới tính (Nam/nữ): [user1_gender]
5. Dân tộc: [user1_ethnicity] 6. Quốc tịch: [user1_nationality]
7. Nơi đăng ký khai sinh: [user1_birth_registration_place_ward]
8. Quê quán: [user1_hometown]
9. Nơi thường trú: [user1_permanent_address]
10. Số CCCD đã được cấp (3): [user1_id_number] tại [user1_id_issue_place]
11. Số CMND đã được đã được cấp (4): [user1_id_number]
Đề nghị (1):.......... xác nhận số Chứng minh nhân dân và số thẻ Căn cước cước công dân cho tôi. Tôi xin cam đoan những thông tin kê khai trên là đúng sự thật.
    [place], ngày [day]tháng[month] năm[year]
""".replace("..........",'[#another]')

In [7]:
# Function to process a sentence
def process_sentence(sentence):
    '''
    Function to remove all number, special character.
    Just keep again words.
    Input: sentence.
    Output: sentence without number, special character.
    '''
    # Step 1: Convert to lowercase
    sentence = sentence.lower()
    # # Step 2: Remove all numbers
    # sentence = re.sub(r'\d+', '', sentence)
    # Step 3: Keep only words and numbers (remove punctuation)
    sentence = re.sub(r'[^\w\s\/]', '', sentence)
    # Step 4: Strip leading/trailing whitespace and split by spaces
    words = sentence.strip().split()
    return words

def extract_form(words,pattern,match_index=0):
    '''
    Function to extract form by pattern re, and with match_index is index of result.
    Input: words, re partern, match_index to return.
    Output: apply pattern, take matches[match_index]
    '''
    # Find all matches
    matches = pattern.findall(words)
    # Collecting the sentences
    sentences = []
    for match in matches:
        sentence = match[match_index].strip()  # Get the text before ".........."
        sentences.append(process_sentence(sentence))
    return sentences

def calculate_similarity_two_lists(list1, list2):
    '''
    Function to calculate similarity 2 list.
    EX:['tờ', 'khai', 'căn', 'cước', 'công', 'dân', 'họ', 'chữ', 'đệm', 'và', 'tên']
    ['họ', 'chữ', 'đệm', 'và', 'tên', 'gọi', 'khác', 'nếu', 'có']
    GPT recommend new metrics: jaccard similarity.
    '''
    if len(list1) == len(list2) == 0:
        return {
        'exact_match_percentage': 100.0,
        'jaccard_similarity': 100.0
    }
    # Exact matching
    exact_match_count = sum(1 for a, b in zip(list1, list2) if a == b)
    exact_match_percentage = exact_match_count / max(len(list1), len(list2)) * 100
    
    # Jaccard similarity
    set1, set2 = set(list1), set(list2)
    jaccard_similarity = len(set1.intersection(set2)) / len(set1.union(set2)) * 100
    
    return {
        'exact_match_percentage': exact_match_percentage,
        'jaccard_similarity': jaccard_similarity
    }

def fill_tagname_to_form(form, list_tag_name):
    '''
    Function fill list_tagname to form (here is input form, with all #another).
    Ensure that list_tag_name len is same with len of #another
    Parameters:
    form (str): The input form with #another placeholders.
    list_tag_name (list of lists): The list of tagnames to fill in.
    
    Returns:
    str: The form with tagnames filled in.
    '''
    # Ensure the length matches the number of placeholders
    num_placeholders = form.count('[#another]')
    if len(list_tag_name) != num_placeholders:
        raise ValueError(f"Mismatch: Found {num_placeholders} placeholders, but {len(list_tag_name)} tagnames were provided.")
    
    # Replace each occurrence of [#another] with the corresponding tagname
    for tag in list_tag_name:
        form = form.replace('[#another]', f'[{tag[0]}]', 1)
    
    return form

def fill_label_to_input_form(input_form, label_form):
    '''
    Function to fill from label form to input form (fill tagname into).
    Input: input form with all ......., label form with [#tagname] is placed.
    '''
    pattern = re.compile(r'(.*?)(\[[^\d].+?\])', re.DOTALL)
    # Find all matches input
    input_sentences = extract_form(input_form,pattern)
    # Find all matches label
    label_sentences = extract_form(label_form,pattern)
    label_tagnames = extract_form(label_form,pattern,match_index=1)

    ## Print testing 
    print(input_sentences)
    print(label_sentences)
    #Filling
    index_label = 0
    tagname_to_fills = []
    for input_sentence in input_sentences:
        while True:
            try:
                jaccard_similarity = calculate_similarity_two_lists(input_sentence,label_sentences[index_label])['jaccard_similarity']
            except:
                print("wrong something")
                return "wrong something"
            if jaccard_similarity > 50.0:
                tagname_to_fills.append(label_tagnames[index_label])
                index_label +=1
                break
            else:
                index_label += 1
    return fill_tagname_to_form(input_form, tagname_to_fills)

print(fill_label_to_input_form(input_form, label_form))

[['cộng', 'hòa', 'xã', 'hội', 'chủ', 'nghĩa', 'việt', 'nam', 'độc', 'lập', 'tự', 'do', 'hạnh', 'phúc', 'giấy', 'đề', 'nghị', 'xác', 'nhận', 'số', 'chứng', 'minh', 'nhân', 'dân', 'kính', 'gửi1'], ['1', 'họ', 'chữ', 'đệm', 'và', 'tên2'], ['2', 'họ', 'chữ', 'đệm', 'và', 'tên', 'gọi', 'khác', 'nếu', 'có2'], ['3', 'ngày', 'tháng', 'năm', 'sinh'], ['/'], ['/'], ['4', 'giới', 'tính', 'nam/nữ'], ['5', 'dân', 'tộc'], ['6', 'quốc', 'tịch'], ['7', 'nơi', 'đăng', 'ký', 'khai', 'sinh'], ['8', 'quê', 'quán'], ['9', 'nơi', 'thường', 'trú'], ['10', 'số', 'cccd', 'đã', 'được', 'cấp', '3'], ['tại', '11', 'số', 'cmnd', 'đã', 'được', 'đã', 'được', 'cấp', '4'], ['đề', 'nghị', '1'], ['xác', 'nhận', 'số', 'chứng', 'minh', 'nhân', 'dân', 'và', 'số', 'thẻ', 'căn', 'cước', 'cước', 'công', 'dân', 'cho', 'tôi', 'tôi', 'xin', 'cam', 'đoan', 'những', 'thông', 'tin', 'kê', 'khai', 'trên', 'là', 'đúng', 'sự', 'thật'], ['ngày'], ['tháng'], ['năm']]
[['cộng', 'hòa', 'xã', 'hội', 'chủ', 'nghĩa', 'việt', 'nam', 'độc', 'l